In [1]:
import os
import ahpy
import json
import math
import random
import hashlib
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import matplotlib.pyplot as plt

from utils import Infiltrometro, ALL_FUNCTIONS, nse, points_distance, USO_SOLO_CLASS, calculate_soil_type, SOIL_TYPES

from tqdm import tqdm
from xgboost import XGBRegressor
from shapely.geometry import Point
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from itertools import combinations, product, islice
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import QuantileTransformer

In [2]:
print("Lendo Rasteres")
clay = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\20_CLAY.tif")
silt = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\20_SILT.tif")
sand = rxr.open_rasterio(r"D:\Mestrado\Trabalho Final\SIG\20_SAND.tif")

Lendo Rasteres


In [3]:
# Calcular o tipo de solo
soil_types = calculate_soil_type(sand.values.flatten(), clay.values.flatten())

Processando Texturas: 100%|██████████| 82791801/82791801 [00:21<00:00, 3774814.96it/s]


In [10]:
# Alterando para o index na lista de tipos de solo
soil_index = {name.upper(): i for i, name in enumerate(SOIL_TYPES) if name is not None}
print("TYPES:", pd.DataFrame({"A":soil_types})["A"].unique())

soil_indices = np.vectorize(lambda s: soil_index.get(s.upper() if s is not None else None, 0))(soil_types)
print("INDICES:", pd.DataFrame({"A":soil_indices})["A"].unique())

# Converter de 1D para 2D com o mesmo shape do raster
soil_array = soil_indices.reshape(sand.shape[1:])

# Criar novo DataArray com metadados do raster original
soil_da = xr.DataArray(
    soil_array,
    dims=("y", "x"),
    coords={"x": sand.x, "y": sand.y},
    name="soil_type"
)

# Aplicar o CRS e garantir compatibilidade espacial
soil_da.rio.write_crs(sand.rio.crs, inplace=True)
soil_da = soil_da.rio.reproject_match(sand)

# Salvar o resultado
soil_da.rio.to_raster(r"D:\Mestrado\Trabalho Final\SIG\20_SOILTYPE.tif")

TYPES: ['Sandy Clay Loam' 'Sandy Loam' None 'Clay Loam' 'Loam' 'Loamy Sand'
 'Sand' 'Sandy Clay' 'Silty Clay Loam' 'Silty Clay' 'Clay' 'Silt Loam']
INDICES: [ 5 12  0  6  9 11 10  4  3  2  1  8]
